# Model Training Notebook

This notebook is for training a custom object detection model using YOLOv8 and Ultralytics.

In [ ]:
# Install necessary libraries
!pip install ultralytics roboflow wandb

In [ ]:
# Import libraries
import os
from ultralytics import YOLO
from roboflow import Roboflow
import wandb # Added for Weights & Biases
from google.colab import drive # Added for Google Drive integration

# Mount Google Drive
drive.mount('/content/drive')

# --- Configuration ---

In [ ]:
# Replace with your Roboflow API key and project details
# ROBOFLOW_API_KEY = "YOUR_ROBOFLOW_API_KEY" # No longer needed as API key is hardcoded below
# ROBOFLOW_WORKSPACE = "YOUR_ROBOFLOW_WORKSPACE"
# ROBOFLOW_PROJECT = "YOUR_ROBOFLOW_PROJECT"
# ROBOFLOW_VERSION = "YOUR_ROBOFLOW_VERSION" # e.g., "1" or "1.0"

# Model configuration
MODEL_NAME = "yolov8m.pt" # or yolov8s.pt, yolov8m.pt, etc.
EPOCHS = 50
IMG_SIZE = 640
BATCH_SIZE = 16
PATIENCE = 10 # Added for early stopping

# Google Drive paths for saving results
# DRIVE_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision/"
    
DRIVE_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision" # Sin barra final
MODEL_FILE_NAME = "best_model_side_project.pt"  
full_model_path = os.path.join(DRIVE_BASE_PATH, MODEL_FILE_NAME)
DATASET_FINAL = "Dataset_final" # Final curated dataset
full_dataset_path = os.path.join(DRIVE_BASE_PATH, DATASET_FINAL)

# Dataset-specific output names (adjust as needed for your two datasets)
DATASET_1_OUTPUT = "dataset_1st_cleaning" # Dataset with cleaning Jorge and my second cleaning
DATASET_2_OUTPUT = "dataset_2nd_cleaning" # Dataset with two Jorge cleanings
DATASET_3_OUTPUT = "dataset_final_output" # Final curated dataset


# Create output directories if they don't exist
os.makedirs(os.path.join(DRIVE_BASE_PATH, DATASET_1_OUTPUT), exist_ok=True)
os.makedirs(os.path.join(DRIVE_BASE_PATH, DATASET_2_OUTPUT), exist_ok=True)
os.makedirs(os.path.join(DRIVE_BASE_PATH, DATASET_3_OUTPUT), exist_ok=True)

# --- Download Dataset from Roboflow ---

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="418MLJVSz2BhvkdjCv19")
# project = rf.workspace("juan-d-computer-vision-projects").project("brand-logo-detector-team-ytebc")
# version = project.version(1)
# dataset = version.download("yolov8")

# The dataset will be downloaded to a directory named after your project version
# e.g., "your_project_name-your_version_number"

# --- Initialize YOLO Model ---

In [ ]:
# Load a pre-trained YOLOv8 model
# model = YOLO(MODEL_NAME)
# Load the model from the specified Drive path
model = YOLO(full_model_path)

# --- Weights & Biases Integration ---

In [ ]:
# Log in to Weights & Biases (you'll be prompted to enter your API key if not already logged in)
wandb.login()

# Initialize a W&B run
wandb.init(
    project="brand-logo-detection", # Replace with your W&B project name
    config={
        "epochs": EPOCHS,
        "imgsz": IMG_SIZE,
        "batch": BATCH_SIZE,
        "model_name": MODEL_NAME,
        "patience": PATIENCE
    }
)

# --- Train the Model ---

In [ ]:
# The data argument should point to the dataset.yaml file downloaded by Roboflow
# This file is usually located in the downloaded dataset directory.
# Example: os.path.join(dataset.location, "data.yaml")
results = model.train(
    data=os.path.join(full_dataset_path, "data.yaml"),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE, # Early stopping: stop training if no improvement for PATIENCE epochs
    # Learning rate scheduling is handled automatically by Ultralytics' optimizer
    # You can customize optimizer and LR scheduler in train() arguments if needed
    # e.g., optimizer='Adam', lr0=0.01, lrf=0.001
    project=os.path.join(DRIVE_BASE_PATH, DATASET_3_OUTPUT), # Base directory for saving runs
    name='train' # Specific folder name for this run (e.g., 'train', 'train2', etc.)
)

# --- Evaluate the Model ---

In [ ]:
metrics = model.val() # Evaluate the model on the validation set

In [ ]:
print("\n--- Evaluation Metrics ---")
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")
# You can access other metrics like:
# metrics.box.maps # list of mAPs for each class
# metrics.speed # inference speed metrics

# --- Save the Model ---

In [ ]:
# Ultralytics automatically saves the best model as 'best.pt' and the last model as 'last.pt'
# in 'runs/detect/trainX/weights/' directory (where X is the run number).
# To load a saved model:
# loaded_model = YOLO('runs/detect/trainX/weights/best.pt')
# You can also export it to other formats if needed
# model.export(format="onnx") # export the model to ONNX format

print("Model training and evaluation complete!")

# --- Finish Weights & Biases Run ---

In [ ]:
wandb.finish()